In [1]:
# Visualization
# # Modified from https://www.kaggle.com/code/colinlagator/play-animation-create-gifs-in-python

from PIL import Image, ImageDraw, ImageFont
import numpy as np
import copy

PLAYER_SIZE = 10  # Radius or width/2 of player markers.
ORIENTATION_OFFSET = (
    PLAYER_SIZE + 4
)  # So that the orientation arc is not right on top of the player marker.
SPEED_FACTOR = 3  # Scales speed vector to be more visible.
HIGHLIGHT_PADDING = 2
FOOTBALL_WIDTH = 10
FOOTBALL_HEIGHT = 6

colors = {
    "offense": "#D50A0A",
    "defense": "#4B92DB",
    "football": "#825736",
    "speed": "#000000",
    "orientation": "#000000",
    "field": "#B1DEB5",
    "yardline": "#FFFFFF",
    "yard_marker": "#000000",
    "highlight": "#FFFFFF",
    "player_text": "#FFFFFF",
}


def get_blank_field():
    yardlines = np.arange(100, 1100 + 1, 100)
    yardline_width = 4

    yard_mark = (
        list(np.arange(0, 50, 10)) + [50] + list(reversed(list(np.arange(0, 50, 10))))
    )
    font_size = 40

    # Draw a green rectangle
    field = Image.new("RGB", (1200, 533), colors["field"])
    draw = ImageDraw.Draw(field)

    # Draw the yardlines and the yard marker text
    assert yardline_width % 2 == 0
    for yl, ym in zip(yardlines, yard_mark):
        yl_x = yl - (yardline_width / 2)
        draw.line(
            [(yl_x, 0), (yl_x, 533)], width=yardline_width, fill=colors["yardline"]
        )

        font = ImageFont.load_default(size=font_size)
        draw.text(
            (yl - (font_size / 2), 533 - (font_size + 5)),
            str(ym),
            font=font,
            fill=colors["yard_marker"],
        )

    return field


def draw_play_frame(frame, pos_team, players, highlight_ids=[]):
    field = get_blank_field()
    draw = ImageDraw.Draw(field)

    # Not sure this actaully speeds anything up, but it made it easier to write.
    # Note that the 'football' entry has a NaN for nflId.
    frame_dict = frame.set_index("nflId").to_dict("index")

    for nfl_id, player in frame_dict.items():
        x = round(player["x"], 1) * 10
        y = round(player["y"], 1) * 10

        # Draw a shadow behind any players to be highlighted
        if nfl_id in highlight_ids:
            highlight_box = (
                (x - PLAYER_SIZE) - HIGHLIGHT_PADDING,
                (y - PLAYER_SIZE) - HIGHLIGHT_PADDING,
                (x + PLAYER_SIZE) + HIGHLIGHT_PADDING,
                (y + PLAYER_SIZE) + HIGHLIGHT_PADDING,
            )
            if player["club"] == pos_team:
                draw.ellipse(highlight_box, fill=colors["highlight"])
            else:
                draw.rectangle(highlight_box, fill=colors["highlight"])
        # Draw the football
        # TODO: Set a Z-Index so that the football is always on top
        if player["club"] == "football":
            draw.ellipse(
                (
                    x - FOOTBALL_WIDTH,
                    y - FOOTBALL_HEIGHT,
                    x + FOOTBALL_WIDTH,
                    y + FOOTBALL_HEIGHT,
                ),
                fill=colors[player["club"]],
            )
        # Draw the players.
        else:
            player_info = players[nfl_id]

            # Orientation
            # Angles from tracking_df are CW from 0 north.
            # Pillow angles are CCW from 0 east.
            # 90-tracking_df_angle=pillow_angle is the conversion.
            draw.arc(
                (
                    x - ORIENTATION_OFFSET,
                    y - ORIENTATION_OFFSET,
                    x + ORIENTATION_OFFSET,
                    y + ORIENTATION_OFFSET,
                ),
                (90 - player["o"] - 25) % 360,
                (90 - player["o"] + 25) % 360,
                fill=colors["orientation"],
                width=2,
            )
            # Speed (pie slice seemed easier than a line to do the angle)
            speed = player["s"] * SPEED_FACTOR
            draw.pieslice(
                (
                    x - PLAYER_SIZE - speed,
                    y - PLAYER_SIZE - speed,
                    x + PLAYER_SIZE + speed,
                    y + PLAYER_SIZE + speed,
                ),
                (90 - player["dir"] - 1) % 360,
                (90 - player["dir"] + 1) % 360,
                fill=colors["speed"],
                width=2,
            )
            if player["club"] == pos_team:
                draw.ellipse(
                    (
                        x - PLAYER_SIZE,
                        y - PLAYER_SIZE,
                        x + PLAYER_SIZE,
                        y + PLAYER_SIZE,
                    ),
                    fill=colors["offense"],
                )
            else:
                draw.rectangle(
                    (
                        x - PLAYER_SIZE,
                        y - PLAYER_SIZE,
                        x + PLAYER_SIZE,
                        y + PLAYER_SIZE,
                    ),
                    fill=colors["defense"],
                )
            font = ImageFont.load_default()
            draw.text(
                (x, y, x + PLAYER_SIZE * 2, y + PLAYER_SIZE * 2),
                str(player_info["position"]),
                font=font,
                fill="white",
                anchor="mm",
            )

    return field


def finalize(field, min_x=None, max_x=None):
    """
    Finalizes the image. Does the following
    - Optionally crops out empty field according to min_x, max_x
    """
    if (min_x is not None) & (max_x is not None):
        field = field.crop((min_x, 0, max_x, 533))

    return field


def create_play_gif(
    game_id,
    play_id,
    tracking_df,
    plays_df,
    players_df,
    gif_name,
    crop=False,
    highlight_ids=[],
):
    """
    Draws the play frame by frame and saves to gif

    Parameters
    game_id - The gameId of the play
    play_id - The playId of the play
    tracking_df - A df of player_tracking_data that contains
    a unique gameId and a unique playId
    plays_df - A df of plays that contains a unique gameId and a unique playId
    players_df - A df of players that contains a unique nflId
    gif_name - The name of the gif, minus the .gif extension. This is
    added automatically.
    crop - Whether or not to crop the gif to only contain the minimum and
    maximum x values within the entire play
    highlight_ids - The ids of players to draw a shadow behind in
    order to call attention to them.

    """
    game_df = tracking_df[tracking_df["gameId"] == game_id]
    play_df = game_df[game_df["playId"] == play_id]
    if play_df.shape[0] == 0:
        raise ValueError("no play found with the given game_id and play_id")

    min_x = (round(play_df.x.min(), 1) * 10) - 50
    max_x = (round(play_df.x.max(), 1) * 10) + 50

    gif_frames = []
    frames = play_df["frameId"].values
    # TODO: This is a hacky way to get the pos_team. Fix this. Lessen reads.
    pos_team = plays_df[plays_df["playId"] == play_df["playId"].values[0]][
        "possessionTeam"
    ].values[0]
    # TODO: This is a hacky way to get the players. Fix this. Don't need all of them.
    players = players_df.set_index("nflId").to_dict("index")
    for i, frame_id in enumerate(frames):
        print(
            gif_name,
            ": Generating Frames",
            str(((i + 1) / len(frames)) * 100)[:5] + "%",
            end="\r",
        )
        frame = play_df[play_df["frameId"] == frame_id].copy(deep=True)
        field = get_blank_field()

        field = draw_play_frame(frame, pos_team, players, highlight_ids)

        if crop:
            field = finalize(field, min_x=min_x, max_x=max_x)
        else:
            field = finalize(field)

        gif_frames.append(field)
    print(
        gif_name,
        ":",
        "Compressing and Saving (This can take as long as generating the frames)",
    )
    frame_one = gif_frames[0]
    frame_one.save(
        f"{gif_name}.gif",
        format="GIF",
        append_images=gif_frames[1:],
        save_all=True,
        duration=100,
        loop=0,
    )
    print(gif_name, ":", "Done")

In [2]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("max_colwidth", None)
import numpy as np
import os

In [5]:
# Data Importing
DATA_ROOT = "../data/"
tracking_data_parts = {}
for i in range(1, 10):
    tracking_data_parts[i] = pd.read_csv(
        os.path.join(DATA_ROOT, f"tracking_week_{i}.csv")
    )
tracking_data = pd.concat(tracking_data_parts.values(), ignore_index=True)
plays_data = pd.read_csv(os.path.join(DATA_ROOT, "plays.csv"))
players_data = pd.read_csv(os.path.join(DATA_ROOT, "players.csv"))

In [4]:
game_id = 2018090600
play_id = 75

create_play_gif(
    game_id,
    play_id,
    tracking_data,
    plays_data,
    players_data,
    "../gifs/" + str(game_id) + "_" + str(play_id),
)

ValueError: no play found with the given game_id and play_id